# Jobs Data Prep

## Libraries

In [2]:
import numpy as np
import pandas as pd
import xlrd
import os
import sys

In [4]:
np.set_printoptions(threshold=np.inf)
pd.options.display.float_format = '{:.4f}'.format

path = 'C:/Users/delightb/Desktop/Final_Project/Senior-Project/Pickle'
####   = pd.read_pickle(os.path.join(path,'abvHouse.pkl'))

# Raw Jobs Data into dataframe


In [ ]:
#xl2 = pd.ExcelFile("/content/drive/My Drive/Colab Notebooks/boise_msa_jobs_2000_to_2018.xlsx")
xl2 = pd.ExcelFile("./Data/boise_msa_jobs_2000_to_2018.xlsx")
rawJobs = xl2.parse("raw")
rawJobs.to_pickle("rawJobs.pkl")

In [ ]:
rawJobs.group.unique()

### From rawJobs.pkl to metro_jobs from pickle

In [ ]:
# open the pickle and turn it into a dataframe
rawJobs = pd.read_pickle("rawJobs.pkl")
metroJobs = rawJobs.copy()

# Splitting Strings
metroJobs['Occ_Cat_Code'], metroJobs['Occ_Sub_Cat_Code'] = metroJobs['occ_code'].str.split('-',1).str


# fixing the ['group'] column
metroJobs['group'] = metroJobs['group'].replace(np.nan,'detailed', regex=True)
metroJobs['occ_titl'] = metroJobs['occ_titl'].replace('Industry Total', 'All Occupations', regex = True)
metroJobs.loc[metroJobs.occ_titl == 'All Occupations', 'group'] = 'total'




# Assigning New Column Names
metroJobs = metroJobs.rename(columns ={'occ_titl': 'Occ_Title', 'group':'Cat_Type',
                                        'tot_emp': 'Emp_Count', 'h_mean':'Hr_Wage_Mean',
                                        'a_mean':'An_Wage_Mean', 'h_median': 'Hr_Median',
                                        'a_median':'An_Median','year':'Year'})



metroJobs['Year'] = metroJobs['Year'].astype(str)
metroJobs['Year'] = metroJobs['Year'].astype('datetime64[Y]')



# Assigning New Data Types
float_list = ['Emp_Count','Hr_Wage_Mean','An_Wage_Mean','Hr_Median','An_Median']

# Cleaning Column Strings for Data Typing
metroJobs[float_list] = metroJobs[float_list].replace('\W','',regex=True)

# Assigning new Data Types
#         need to use floats becuase int cannot handle missing
metroJobs = metroJobs.astype({ 'Cat_Type' : 'category',
                               'Occ_Cat_Code':'category','Occ_Sub_Cat_Code': 'category'})

metroJobs[float_list] = metroJobs[float_list].apply(pd.to_numeric, errors = 'coerce')


# Dropped Columns
metroJobs = metroJobs.drop(['prim_state','area','area_name','occ_code',
                              'emp_prse','mean_prse','h_wpct10','h_wpct25',
                             'h_wpct75','h_wpct90','a_wpct10','a_wpct25',
                             'a_wpct75','a_wpct90'], axis = 1)

# Assigning New Order to the Columns
metroJobs = metroJobs[['Year','Occ_Cat_Code','Occ_Sub_Cat_Code','Occ_Title','Cat_Type',
                        'Emp_Count','Hr_Wage_Mean','An_Wage_Mean','Hr_Median','An_Median']]

metroJobs['Occ_Title'] = metroJobs['Occ_Title'].str.title()
metroJobs['Emp_Count'] = metroJobs['Emp_Count'].fillna(0)
metroJobs['An_Wage_Mean'] = metroJobs['An_Wage_Mean'].fillna(0)



metroJobs.to_pickle('metroJobs.pkl')
metroJobs.head(5)

# Clean jobs for empolyee count for job type using major job categories

In [ ]:
metroJobs = pd.read_pickle('metroJobs.pkl')

# start filtering to make df with Occ_Title as cols and the value be Emp_Count for each year.

# 0000 is the sub_cat # for the head of Cat
jobsF = metroJobs.copy()
jobsF = jobsF.loc[jobsF['Cat_Type'] != 'detailed']


jobsF['Occ_Title'] = jobsF['Occ_Title'].replace('Services','Service', regex = True)
jobsF['Occ_Title'] = jobsF['Occ_Title'].replace('Practitioners','Practitioner', regex = True)
jobsF['Occ_Title'] = jobsF['Occ_Title'].replace(' Mathematical Science Occupations',' Mathematical Occupations', regex = True)


jobsClean = jobsF.copy()
jobsClean.to_pickle('jobsClean.pkl')
jobsClean.head()


## Create employee count % change

In [ ]:
df = pd.read_pickle('jobsClean.pkl')
df = df[['Year','Occ_Title','Emp_Count']]

df['Job_Emp_Change'] = df.groupby('Occ_Title')['Emp_Count'].pct_change()

df = pd.pivot(df, values = 'Job_Emp_Change', index = 'Year', columns = 'Occ_Title')
df = df.reset_index()

df.at[6,'Construction And Extraction Occupations'] = np.nan

df.to_pickle('jobChange.pkl')
df.head()

In [5]:
df = pd.read_pickle(os.path.join(path,'jobChange.pkl'))

print(df.isnull().sum())

Occ_Title
Year                                                          0
All Occupations                                               2
Architecture And Engineering Occupations                      1
Arts, Design, Entertainment, Sports, And Media Occupations    1
Building And Grounds Cleaning And Maintenance Occupations     1
Business And Financial Operations Occupations                 1
Community And Social Service Occupations                      1
Computer And Mathematical Occupations                         1
Construction And Extraction Occupations                       2
Education, Training, And Library Occupations                  3
Farming, Fishing, And Forestry Occupations                    1
Food Preparation And Serving Related Occupations              1
Healthcare Practitioner And Technical Occupations             1
Healthcare Support Occupations                                1
Installation, Maintenance, And Repair Occupations             1
Legal Occupations             

## Jobs by Industry, employee count

In [ ]:
jobsF1 = pd.read_pickle('jobsClean.pkl')
jobsF1 = jobsF.drop(['Occ_Cat_Code', 'Occ_Sub_Cat_Code', 'Cat_Type'], axis =1)
jobsF1 = jobsF1.iloc[:,0:3]

jobsP1 = pd.pivot(jobsF1, values = 'Emp_Count', index = 'Year', columns = 'Occ_Title')
jobsP1 = jobsP1.reset_index()

# Gives the wrong answer, 2018 should be 706140
jobsP1['Job_Manual_Count'] = jobsP1.iloc[:,2:].sum(axis = 1)


# set manually because the 2000 data did not have a sum of all of the occupations
jobsP1.at[0,'All Occupations'] = 229780

jobsP1.to_pickle('jobTypes.pkl')
jobsP1.head()

## Clean jobs for empolyee count for Salery Range not using major job categories

## Jobs by Salery Range, empolyee count

In [ ]:
df = pd.read_pickle('metroJobs.pkl')

df = df.loc[df['Cat_Type'] == 'detailed']
df = df[['Year', 'Emp_Count' ,'An_Wage_Mean']]

jobsSalery = df.groupby([df.Year,pd.cut(df.An_Wage_Mean,[10000,40000,70000,100000,150000,200000,300000]).astype(str)])[['Emp_Count']].sum().reset_index()

jobsSalery = pd.pivot(jobsSalery, values = 'Emp_Count', index = 'Year', columns = 'An_Wage_Mean')
jobsSalery = jobsSalery.reset_index()
jobsSalery = jobsSalery.replace(np.nan,0)

# fix the order of the columns
jobsSalery = jobsSalery[['Year','(10000.0, 40000.0]','(40000.0, 70000.0]','(70000.0, 100000.0]',
          '(100000.0, 150000.0]', '(150000.0, 200000.0]', '(200000.0, 300000.0]' ]]

jobsSalery.to_pickle('jobSalery.pkl')
jobsSalery

## Percent change in # of emp for wage range

In [ ]:
df = pd.read_pickle('metroJobs.pkl')

df = df.loc[df['Cat_Type'] == 'detailed']
df = df[['Year', 'Emp_Count' ,'An_Wage_Mean']]

df = df.groupby([df.Year,pd.cut(df.An_Wage_Mean,[10000,40000,70000,100000,150000,200000,
                                                 300000]).astype(str)])[['Emp_Count']].sum().reset_index()

df['Salery_Count_Change'] = df.groupby('An_Wage_Mean')['Emp_Count'].pct_change()

df = pd.pivot(df, values = 'Salery_Count_Change', index = 'Year', columns = 'An_Wage_Mean')
df = df.reset_index()

# These are the first values for that range, nothing to compare against so NaN
df.at[5, '(150000.0, 200000.0]'] = np.nan
df.at[9, '(200000.0, 300000.0]'] = np.nan

# Not sure what to do when values go from 220 - 0 and 0 - 220, setting to NaN for now.
#df.at[16, '(150000.0, 200000.0]'] = np.nan
df.at[18, '(150000.0, 200000.0]'] = np.nan

df = df[['Year','(10000.0, 40000.0]','(40000.0, 70000.0]','(70000.0, 100000.0]',
          '(100000.0, 150000.0]', '(150000.0, 200000.0]', '(200000.0, 300000.0]' ]]

df.to_pickle('salChange.pkl')
df

In [ ]:
job1 = pd.read_pickle('jobTypes.pkl')
job2 = pd.read_pickle('jobSalery.pkl')

jobFeatures = pd.merge(job1, job2, on = 'Year', how = 'left')
jobFeatures = jobFeatures.drop(['Job_Manual_Count'], axis = 1)

jobFeatures.to_pickle('jobFeatures.pkl')
jobFeatures.info()

In [13]:
df1 = pd.read_pickle(os.path.join(path,'jobChange.pkl'))
df2 = pd.read_pickle(os.path.join(path,'salChange.pkl'))

df = pd.merge(df1, df2, on = 'Year', how = 'left')

# replace nan values in salery columns with 0
df['(150000.0, 200000.0]'].fillna(0, inplace = True)
df['(200000.0, 300000.0]'].fillna(0, inplace = True)
df['(100000.0, 150000.0]'].fillna(0, inplace = True)

df.fillna(df.mean(), inplace=True)

df.to_pickle(os.path.join(path,'jobShift.pkl'))
df


,Year,All Occupations,Architecture And Engineering Occupations,"Arts, Design, Entertainment, Sports, And Media Occupations",Building And Grounds Cleaning And Maintenance Occupations,Business And Financial Operations Occupations,Community And Social Service Occupations,Computer And Mathematical Occupations,Construction And Extraction Occupations,"Education, Training, And Library Occupations",...,Production Occupations,Protective Service Occupations,Sales And Related Occupations,Transportation And Material Moving Occupations,"(10000.0, 40000.0]","(40000.0, 70000.0]","(70000.0, 100000.0]","(100000.0, 150000.0]","(150000.0, 200000.0]","(200000.0, 300000.0]"
1,2001-01-01,0.0227,-0.1135,-0.0916,0.0053,-0.0554,-0.0176,-0.0581,-0.0022,-0.1151,...,-0.1350,-0.0052,-0.0250,-0.0455,-0.0947,0.3027,0.2183,0.0000,0.0000,0.0000
2,2002-01-01,0.0167,0.0421,0.0444,0.0447,0.0388,0.0717,0.0917,-0.1344,0.0283,...,-0.0540,0.0757,0.0420,0.0260,-0.0472,-0.1150,0.2942,0.0000,0.0000,0.0000
3,2003-01-01,0.0112,-0.4099,-0.0193,-0.0214,-0.0277,0.0936,-0.0905,0.0263,0.0283,...,0.1123,0.0631,0.1000,0.0000,0.0219,0.0079,-0.1521,0.0545,0.0000,0.0000
5,2005-01-01,0.0918,0.0561,-0.0774,-0.0427,0.0706,-0.0625,0.0582,-1.0000,0.0411,...,0.1604,-0.0534,0.0302,0.0636,0.0324,0.2038,-0.2977,0.5123,0.0000,0.0000
6,2006-01-01,0.0561,0.1593,0.1573,0.1142,0.1290,0.1111,-0.0169,-0.0616,-0.0042,...,0.0050,-0.0960,0.0505,0.1055,0.0826,-0.0546,1.4439,0.2932,-0.0909,0.0000
7,2007-01-01,0.0579,-0.0525,0.3384,0.0718,-0.0129,0.1886,0.0516,0.0209,0.0152,...,0.1105,0.1155,0.1462,0.1812,0.0448,0.1992,1.0643,-0.0050,0.2000,0.0000
8,2008-01-01,0.0068,0.0212,0.0451,-0.0396,-0.0044,0.3173,-0.0559,-0.1956,-0.0008,...,-0.0294,0.1863,0.0460,-0.0427,-0.0275,0.0656,0.1315,0.1848,0.0417,0.0000
9,2009-01-01,-0.0771,-0.0820,0.0713,-0.0458,-0.0664,0.0000,0.0954,-0.1917,0.0466,...,-0.2573,-0.0558,-0.0818,-0.0527,-0.1012,0.0118,0.0096,-0.7265,-0.0800,0.0000
10,2010-01-01,-0.0315,-0.1484,-0.1754,-0.0504,0.0337,0.0000,0.1319,-0.1266,0.1557,...,-0.0904,-0.0370,-0.0759,-0.1402,0.0485,-0.1189,-0.0593,-0.1953,1.0000,0.8814
11,2011-01-01,-0.0033,0.0384,-0.1711,0.0126,0.0045,-0.0839,-0.0385,-0.0314,-0.0048,...,0.0015,0.0480,0.0056,0.0066,-0.0031,0.0467,0.0009,1.5049,0.1087,-0.4775


In [7]:
df = pd.read_pickle(os.path.join(path,'jobShift.pkl'))
df.head()
#print(df.isnull().sum())

,Year,All Occupations,Architecture And Engineering Occupations,"Arts, Design, Entertainment, Sports, And Media Occupations",Building And Grounds Cleaning And Maintenance Occupations,Business And Financial Operations Occupations,Community And Social Service Occupations,Computer And Mathematical Occupations,Construction And Extraction Occupations,"Education, Training, And Library Occupations",...,Production Occupations,Protective Service Occupations,Sales And Related Occupations,Transportation And Material Moving Occupations,"(10000.0, 40000.0]","(40000.0, 70000.0]","(70000.0, 100000.0]","(100000.0, 150000.0]","(150000.0, 200000.0]","(200000.0, 300000.0]"
0,2000-01-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2001-01-01,nan,-0.1135,-0.0916,0.0053,-0.0554,-0.0176,-0.0581,-0.0022,-0.1151,...,-0.1350,-0.0052,-0.0250,-0.0455,-0.0947,0.3027,0.2183,nan,nan,nan
2,2002-01-01,0.0167,0.0421,0.0444,0.0447,0.0388,0.0717,0.0917,-0.1344,nan,...,-0.0540,0.0757,0.0420,0.0260,-0.0472,-0.1150,0.2942,nan,nan,nan
3,2003-01-01,0.0112,-0.4099,-0.0193,-0.0214,-0.0277,0.0936,-0.0905,0.0263,nan,...,0.1123,0.0631,0.1000,0.0000,0.0219,0.0079,-0.1521,0.0545,nan,nan
4,2004-01-01,0.0201,0.0428,0.2205,0.0231,0.0559,0.0275,0.1901,-0.0074,0.1806,...,-0.1044,0.1553,-0.0134,-0.0431,0.0819,0.0937,0.1512,0.7500,nan,nan
